# Ergonomic pose fine-tuning: 

## fine tuning yolo for our classification task

- option 1: freeze the complete model, use it as a feature extractor and add a classification head to be trained on our dataset
- option 2: cut the keypoint output layer, freeze the rest of the model and add a classification head to be trained on our dataset

### 1) YOLOv8pose model as feature extractor

imports:

In [1]:
# early stopping bei hoher num_epochs (300)
# batch_size festlegen (8, 16?)

# HPO:
# momentum: [0.9, 0.95]
# lr: [1e-4, 5e-5]

In [2]:
from ultralytics import YOLO
import cv2
import numpy as np
from PIL import Image
import torch
from functools import partial

In [3]:
def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    elif torch.backends.mps.is_available():
        return torch.device("mps")
    else:
        return torch.device("cpu")
    
get_device()

device(type='cuda')

### load the model

strangely the yolo model does not like to be frozen and put to eval mode (then it loads some own dataset):

In [4]:
# we had to outcomment this + yolomodel.eval() later
#for param in model.parameters():
#    param.requires_grad = False # freeze the YOLO model

#for name, param in model.named_parameters():
    #print(f"{name} is {'not ' if param.requires_grad else ''}frozen"

defining a classification layer:

In [5]:
import torch
import torch.nn as nn

# `feature_extractor` is the part of the model suitable for feature extraction

class YOLO_kp_Classifier(nn.Module):
    def __init__(self, num_keypoints, num_classes, h1, h2):
        super(YOLO_kp_Classifier, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(num_keypoints*2, h1),
            nn.ReLU(),
            nn.Dropout(0.5)
        )
        self.layer2 = nn.Sequential(
            nn.Linear(h1, h2),
            nn.ReLU(),
            nn.Dropout(0.5)
        )
        self.layer3 = nn.Sequential(
            nn.Linear(h2, h2 // 2),
            nn.ReLU()
        )
        self.dense = nn.Sequential(
            nn.Linear(h2 // 2, num_classes)
        )
    
    def forward(self, keypoints_flattened):
        x = self.layer1(keypoints_flattened)
        x = self.layer2(x)
        x = self.layer3(x)
        output = self.dense(x)
        
        return output

class YOLO_KP_Scorer(nn.Module):
    def __init__(self, num_keypoints, h1, h2):
        super(YOLO_KP_Scorer, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(num_keypoints*2, h1),
            nn.ReLU(),
            nn.Dropout(0.5)
        )
        self.layer2 = nn.Sequential(
            nn.Linear(h1, h2),
            nn.ReLU(),
            nn.Dropout(0.5)
        )
        self.layer3 = nn.Sequential(
            nn.Linear(h2, h2 // 2),
            nn.ReLU()
        )
        self.dense = nn.Sequential(
            nn.Linear(h2 // 2, 1)
        )
    
    def forward(self, keypoints_flattened):
        x = self.layer1(keypoints_flattened)
        x = self.layer2(x)
        x = self.layer3(x)
        output = self.dense(x)
        
        return output

initializing the model:

In [6]:
# initialize the classifier model
num_classes = 3  #  3-class classification problem
num_keypoints = 17  # 17 keypoints in the model

load the own dataset:

In [7]:
from torch.utils.data import random_split, DataLoader, Subset, TensorDataset
from torchvision import datasets, transforms

In [8]:
# Normalization should be applied to the keypoints not the images I think - Vipin

# Custom transform because YOLO likes Numpy Arrays / CV2 Images,
# it automatically resizes it to its preffered size, and converts
# the output back to the source size, better for precision and normalization
from sympy import im


class PILToNumpyTransform:
    def __call__(self, pil_img):
        """
        Convert a PIL Image to an OpenCV Image / Numpy Array.

        Parameters:
            pil_img (PIL.Image): The PIL Image to be converted.

        Returns:
            np.ndarray: The converted OpenCV Image in RGB format.
        """
        # Convert PIL Image to NumPy array (RGB)
        img_array = np.array(pil_img)
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)

        return img_array
    
data_transforms = transforms.Compose([
    transforms.Resize((640, 480)), # Resize images to 640x640
    transforms.ToTensor() # Convert to tensor
])

In [9]:
data_path = 'E:/Users/Vipin\Documents/BHT/3. Semester/Learning from images/Pose Dataset_binned'
own_dataset = datasets.ImageFolder(root=data_path, transform=data_transforms)

In [10]:
own_dataset

Dataset ImageFolder
    Number of datapoints: 118
    Root location: E:/Users/Vipin\Documents/BHT/3. Semester/Learning from images/Pose Dataset_binned
    StandardTransform
Transform: Compose(
               Resize(size=(640, 480), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )

splitting the data into train, validation and test set:

In [11]:
generator1 = torch.Generator().manual_seed(13)  # set seed for reproducibility of the split
train_and_val_dataset, test_dataset = random_split(own_dataset, [0.8, 0.2], generator=generator1)  # 80% training and evaluation, 20% testing

feature extraction with yolo:

In [12]:
def get_keypoints_from_yolo(model, inputs):
    with torch.no_grad(): # gradients are not computed for the frozen model
        results = []
        for input in inputs:
            result = model(input.unsqueeze(0))[0].keypoints.xy[0].to(input.device)
            print(result.shape)
            if result.shape[0] < 17:
                result = torch.cat([result, torch.zeros(17 - result.shape[0], 2).to(input.device)], dim = 0)
            
            results.append(result)
    return torch.stack(results)

processing keypoints:

In [13]:
# z-Value
def process_keypoints_for_classifier(keypoints):
    # Calculate mean and standard deviation with keepdim=True to preserve broadcasting compatibility
    mean_vals = keypoints.mean(dim=1, keepdim=True)
    std_vals = keypoints.std(dim=1, keepdim=True) + 1e-6  # Adding a small epsilon to prevent division by zero

    # Perform standardization
    kp_tensor_standardized = (keypoints - mean_vals) / std_vals
    
    # Flatten the last two dimensions while keeping the batch dimension
    batch_flattened = kp_tensor_standardized.view(keypoints.size(0), -1)
    
    return batch_flattened

#### training the model:

- with cross-entropy loss function (fits our classification task)
- Adam optimizer

HPO:

In [14]:
import optuna
from sklearn.metrics import f1_score, confusion_matrix
import numpy as np

In [15]:
def preprocess_dataset(yolo_model, dataset, batch_size, device):
    processed_keypoints_list = []
    labels_list = []
    
    # Assuming `dataset` is an iterable of (input, label) pairs
    for inputs, labels in dataset:
        inputs = inputs.to(device)
        keypoints = get_keypoints_from_yolo(yolo_model, inputs)  # Extract keypoints        
        processed_kps = process_keypoints_for_classifier(keypoints)  # Process keypoints        
        processed_keypoints_list.append(processed_kps)
        labels_list.append(labels)

    # Convert lists to tensors
    processed_keypoints_tensor = torch.cat(processed_keypoints_list, dim=0)
    labels_tensor = torch.cat(labels_list, dim=0)
    
    # Create a new TensorDataset and DataLoader
    preprocessed_dataset = TensorDataset(processed_keypoints_tensor, labels_tensor)
    preprocessed_loader = DataLoader(preprocessed_dataset, batch_size=batch_size, shuffle=True)
    
    return preprocessed_loader
    
def train_and_eval_model(kp_classifier_model, yolo_model, optimizer, loss_fn, num_epochs, train_loader, val_loader, device):
    
    kp_classifier_model = kp_classifier_model.to(device)  # move model to device
    yolo_model = yolo_model.to(device)  # move YOLO model to device
    kp_classifier_model.train()  # set model to training mode
    
    print("type of train_loader", type(train_loader))

    for epoch in range(num_epochs):
        for batch in train_loader:
            # access images and labels
            inputs = batch[0].to(device)
            labels = batch[1].to(device)
            
            optimizer.zero_grad()  # Zero the parameter gradients
            classification_output = kp_classifier_model(inputs) # get results for the classification
            
            # Use this for classification
            loss = loss_fn(classification_output, labels)
            
            # Use this for scoring
            # loss = loss_fn(classification_output.float(), labels.float())
            
            loss.backward()  # Backpropagate the loss
            optimizer.step()  # Update weights

            # todo: further processing, such as calculating accuracy or loss, goes here

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
            
        
        kp_classifier_model.eval()  # model to evaluation mode

        total_loss = 0
        all_predictions = []
        all_labels = []

        with torch.no_grad():  # no need to compute gradients, because we are in evaluation mode
            for inputs, labels in val_loader:  # iterate over validation dataset
                inputs, labels = inputs.to(device), labels.to(device)  # move data to device
                
                # Not necessary because already preprocessed
                # keypoints = get_keypoints_from_yolo(yolo_model, inputs) # get keypoints from the YOLO model
                # processed_kps = process_keypoints_for_classifier(keypoints) # prepare the keypoints for the classifier
                
                classification_output = kp_classifier_model(inputs) # get results for the classification 
                
                # Use this for classification
                loss = loss_fn(classification_output, labels)
                
                # Use this for scoring
                # loss = loss_fn(classification_output.float(), labels.float())
                
                total_loss += loss.item()  # accumulate the loss
                # get predictions for output
                _, predicted = torch.max(classification_output.data, 1)
                # collect the predictions and labels
                all_predictions.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
            # calculate the validation metrics:
            avg_loss = total_loss / len(val_loader)  # get the average loss
            accuracy = (np.array(all_predictions) == np.array(all_labels)).mean()
            f1 = f1_score(all_labels, all_predictions, average='weighted')  
            conf_matrix = confusion_matrix(all_labels, all_predictions)
            
            print(f"Validation Loss: {avg_loss:.4f}")
            print(f"Validation Accuracy: {accuracy:.4f}")
            print(f"Validation F1 Score: {f1:.4f}")
            print("Confusion Matrix:")
            print(conf_matrix)
            
            # here: F1 score chosen as the metric to optimize
            # other options: - combining metrics like accuracy and F1 score to maximize on both
            #                        - or multi-objective optimization on F1 score and accuracy
    return f1 

In [16]:
# this function was generated by gpt-4

def get_k_fold_indices(n, k=5, random_seed=None):
    """
    Generate indices for k-fold cross-validation.

    Parameters:
    - n: Total number of samples in the dataset.
    - k: Number of folds.
    - random_seed: Optional seed for reproducibility.

    Returns:
    - A list of tuples, each containing (train_indices, val_indices) for a fold.
    """
    # Initialize the random generator
    g = torch.Generator()
    if random_seed is not None:
        g.manual_seed(random_seed)
    
    # Generate a random permutation of indices
    indices = torch.randperm(n, generator=g).tolist()
    
    # Calculate fold sizes
    fold_sizes = [n // k for _ in range(k)]
    for i in range(n % k):
        fold_sizes[i] += 1
    
    # Generate train and validation indices for each fold
    current = 0
    folds_indices = []
    for fold_size in fold_sizes:
        start, end = current, current + fold_size
        val_indices = indices[start:end]
        train_indices = indices[:start] + indices[end:]
        folds_indices.append((train_indices, val_indices))
        current = end
    
    return folds_indices


In [17]:
def objective(trial, dataset, yolo_model):
    # Define hyperparameters to optimize
    lr = trial.suggest_categorical("lr", [1e-3, 1e-4, 5e-5])
    h1 = trial.suggest_categorical("h1", [256, 512, 1024])
    h2 = trial.suggest_categorical("h2", [256, 512, 1024])
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])
    num_epochs = trial.suggest_categorical("num_epochs", [100, 200, 300])

    validation_scores = []

    n = len(dataset)
    k = 5
    folds_indices = get_k_fold_indices(n, k, random_seed=13)
    device = get_device()
    
    for fold, (train_idx, val_idx) in enumerate(folds_indices, start=1):
        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)
        print("type of train subset: " , type(train_subset))
        
        # Create data loaders for the iteration in preprocessing with yolo
        # Batch size 1 because preprocessing must be done image by image to extract the correct keypoints
        train_loader = DataLoader(train_subset, batch_size=1, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=1, shuffle=False)
        
        # Batch size goes here
        train_loader = preprocess_dataset(yolo_model, train_loader, batch_size, device)
        val_loader = preprocess_dataset(yolo_model, val_loader, batch_size, device)
        print("type of train loader: " , type(train_loader))
        
        # Initializing the model and optimizer with the chosen hyperparameters
        # kp_model = YOLO_KP_Scorer(num_keypoints, h1, h2).to(device)
        kp_model = YOLO_kp_Classifier(num_keypoints=num_keypoints, num_classes=num_classes, h1=h1, h2=h2).to(device)
        optimizer = torch.optim.Adam(kp_model.parameters(), lr=lr, weight_decay=1e-5)
    
        # training and evaluating the model - Watch out for the Loss - CrossEntropyLoss for classification and MSELoss for scoring
        validation_score = train_and_eval_model(kp_model, yolo_model, optimizer, nn.CrossEntropyLoss(), num_epochs, train_loader, val_loader, device)
        validation_scores.append(validation_score)

    return np.mean(validation_scores)


In [ ]:
yolo_model = YOLO('yolov8n-pose.pt')
optimize = partial(objective, dataset = train_and_val_dataset, yolo_model = yolo_model)
search_space = {
    "lr": [1e-3, 1e-4, 5e-5],
    "h1": [256, 512, 1024],
    "h2": [256, 512, 1024],
    "batch_size": [4, 8, 16],
    "num_epochs": [100, 200, 300]
}

study = optuna.create_study(sampler=optuna.samplers.GridSampler(search_space), direction='maximize')
study.optimize(optimize, n_trials=243)

# Best hyperparameters
print("Best hyperparameters:", study.best_params)

In [19]:
best_params = study.best_params
best_params

{'lr': 0.0001, 'h1': 512, 'h2': 1024, 'batch_size': 4, 'num_epochs': 300}

In [20]:
best_params = study.best_params
# extract best_parameters
batch_size = best_params['batch_size']
lr = best_params['lr']
h1 = best_params['h1']
h2 = best_params['h2']
num_epochs = best_params['num_epochs']

#### train and evaluate final model on test set:

Unfortunately Optuna cannot output the best model, so we train again on the combined train and validation set with the best parameters found.

dataloader for final evaluation:

In [21]:
train_and_eval_loader = DataLoader(train_and_val_dataset, batch_size=1, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4, pin_memory=True)

train_and_eval_loader = preprocess_dataset(yolo_model, train_and_eval_loader, batch_size, get_device())
test_loader = preprocess_dataset(yolo_model, test_loader, batch_size, get_device())


0: 640x480 (no detections), 9.0ms
Speed: 0.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)
torch.Size([0, 2])

0: 640x480 1 person, 6.5ms
Speed: 0.0ms preprocess, 6.5ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 480)
torch.Size([17, 2])

0: 640x480 1 person, 6.5ms
Speed: 0.0ms preprocess, 6.5ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 480)
torch.Size([17, 2])

0: 640x480 (no detections), 6.5ms
Speed: 0.0ms preprocess, 6.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)
torch.Size([0, 2])

0: 640x480 1 person, 7.0ms
Speed: 0.0ms preprocess, 7.0ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 480)
torch.Size([17, 2])

0: 640x480 1 person, 7.0ms
Speed: 0.0ms preprocess, 7.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)
torch.Size([17, 2])

0: 640x480 1 person, 7.5ms
Speed: 0.0ms preprocess, 7.5ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 480)


final evaluation on test set:

In [23]:
# new instance of the model:
# kp_model = YOLO_KP_Scorer(17, h1, h2)
kp_model = YOLO_kp_Classifier(num_keypoints, num_classes, h1, h2)
kp_model = kp_model.to(get_device())  # move model to device
kp_model.train()  # set model to training mode

# loss function and optimizer
criterion = nn.CrossEntropyLoss() #nn.MSELoss()
optimizer = torch.optim.Adam(kp_model.parameters(), lr=lr, weight_decay=1e-5)

for epoch in range(num_epochs):
    for inputs, labels in train_and_eval_loader:  # Assuming data_loader is your DataLoader instance for the dataset
        inputs, labels = inputs.to(get_device()), labels.to(get_device())  # move data to device
        optimizer.zero_grad()  # Zero the parameter gradients
        classification_output = kp_model(inputs) # get results for the classification 
        
        # Use this for classification
        loss = criterion(classification_output, labels)
        
        # Use this for scoring
        # loss = criterion(classification_output.float(), labels.float())
        
        loss.backward()  # Backpropagate the loss
        optimizer.step()  # Update weights

        # todo: further processing, such as calculating accuracy or loss, goes here

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
        
    
kp_model.eval()  # model to evaluation mode

total_loss = 0
all_predictions = []
all_labels = []
with torch.no_grad():  # no need to compute gradients, because we are in evaluation mode
    for inputs, labels in test_loader:  # iterate over validation dataset
        inputs, labels = inputs.to(get_device()), labels.to(get_device())  # move data to device
        classification_output = kp_model(inputs) # get results for the classification 
        
        # Use this for classification
        loss = criterion(classification_output, labels)
        
        # Use this for scoring
        # loss = criterion(classification_output.float(), labels.float())
        
        total_loss += loss.item()  # accumulate the loss
        _, predicted = torch.max(classification_output.data, 1)
        # collect the predictions and labels
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
    # calculate the validation metrics:
    avg_loss = total_loss / len(test_loader)  # get the average loss
    accuracy = (np.array(all_predictions) == np.array(all_labels)).mean()
    f1 = f1_score(all_labels, all_predictions, average='weighted')  
    conf_matrix = confusion_matrix(all_labels, all_predictions)
    
    print(f"Test Loss: {avg_loss:.4f}")
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Test F1 Score: {f1:.4f}")
    print("Confusion Matrix:")
    print(conf_matrix)

Epoch [1/300], Loss: 1.0472
Epoch [2/300], Loss: 1.1228
Epoch [3/300], Loss: 1.1898
Epoch [4/300], Loss: 0.9125
Epoch [5/300], Loss: 0.7267
Epoch [6/300], Loss: 1.0057
Epoch [7/300], Loss: 0.5941
Epoch [8/300], Loss: 1.0955
Epoch [9/300], Loss: 0.5706
Epoch [10/300], Loss: 1.4644
Epoch [11/300], Loss: 1.0247
Epoch [12/300], Loss: 0.8891
Epoch [13/300], Loss: 0.8012
Epoch [14/300], Loss: 1.3533
Epoch [15/300], Loss: 0.8050
Epoch [16/300], Loss: 0.9901
Epoch [17/300], Loss: 0.6548
Epoch [18/300], Loss: 0.9349
Epoch [19/300], Loss: 1.0869
Epoch [20/300], Loss: 0.8145
Epoch [21/300], Loss: 0.8749
Epoch [22/300], Loss: 1.4428
Epoch [23/300], Loss: 0.6674
Epoch [24/300], Loss: 1.4872
Epoch [25/300], Loss: 0.5583
Epoch [26/300], Loss: 0.3842
Epoch [27/300], Loss: 1.0396
Epoch [28/300], Loss: 0.9086
Epoch [29/300], Loss: 0.6882
Epoch [30/300], Loss: 1.0397
Epoch [31/300], Loss: 0.4248
Epoch [32/300], Loss: 0.7491
Epoch [33/300], Loss: 0.5157
Epoch [34/300], Loss: 0.4379
Epoch [35/300], Loss: 0